## **Scraping data about LOTR characters**

### Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import seaborn as sns

### Connection

In [3]:
url = "https://lotr.fandom.com/wiki/Main_Page"
response = requests.get(url)
response.status_code

200

In [47]:
PATH = 'C:\webdrivers\chromedriver.exe'
driver = webdriver.Chrome(executable_path=PATH)
driver.get(url)
driver.implicitly_wait(10)
cookies = driver.find_element(By.XPATH,'//div[@class="NN0_TB_DIsNmMHgJWgT7U XHcr6qf5Sub2F2zBJ53S_"]')
cookies.click()
good_characters = driver.find_element(By.XPATH,'((//tbody)[1]//tr[3]//td//p//a)[1]').get_attribute('href')
bad_characters = driver.find_element(By.XPATH,'((//tbody)[1]//tr[3]//td//p//a)[2]').get_attribute('href')
all_links = []
character_data = []
good_characters_names = []
bad_characters_names = []

C:\Users\piotr\AppData\Local\Temp\ipykernel_12160\1560111238.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=PATH)


### Getting good characters names in a list

In [48]:
driver.get(good_characters)
driver.implicitly_wait(5)
character_links = driver.find_elements(By.XPATH,'(//tbody)[1]//div[@class="floatnone"]//a')
for i in character_links:
    link = i.get_attribute('href')
    link_str_split = link.split('/',4)[-1]
    link_str_split = link_str_split.replace('Th%C3%A9oden','Théoden')
    link_str_split = link_str_split.replace('%C3%89omer','Éomer')
    link_str_split = link_str_split.replace('%C3%89owyn','Éowyn')
    link_str_split = link_str_split.replace('_',' ')
    good_characters_names.append(link_str_split)

In [49]:
good_characters_names

['Frodo Baggins',
 'Sam Gamgee',
 'Meriadoc Brandybuck',
 'Peregrin Took',
 'Gandalf',
 'Aragorn',
 'Legolas',
 'Gimli',
 'Boromir',
 'Tom Bombadil',
 'Elrond',
 'Galadriel',
 'Théoden',
 'Éomer',
 'Éowyn',
 'Bilbo',
 'Faramir',
 'Denethor',
 'Thranduil',
 'Radagast',
 'Arwen',
 'Glorfindel',
 'Beorn',
 'Thorin',
 'Bard']

### Getting bad characters names in a list

In [50]:
driver.get(bad_characters)
driver.implicitly_wait(5)
character_links = driver.find_elements(By.XPATH,'(//tbody)[1]//div[@class="floatnone"]//a')
for i in character_links:
    link = i.get_attribute('href')
    link_str_split = link.split('/',4)[-1]
    link_str_split = link_str_split.replace('Kham%C3%BBl','Khamûl')
    link_str_split = link_str_split.replace('Durin%27s_Bane',"""Durin's Bane""")
    link_str_split = link_str_split.replace('_',' ')
    bad_characters_names.append(link_str_split)

In [51]:
bad_characters_names

['Morgoth Bauglir',
 'Sauron',
 'The Witch-king',
 'Khamûl',
 'Saruman',
 'Gollum',
 "Durin's Bane",
 'Gothmog',
 'Gothmog (Lieutenant of Morgul)',
 'The Great Goblin',
 'Grima',
 'Ungoliant',
 'Shelob',
 'Watcher in the Water',
 'Smaug']

### Getting all necessary links

In [52]:
for l in [good_characters,bad_characters]:
    driver.get(l)
    driver.implicitly_wait(5)
    character_links = driver.find_elements(By.XPATH,'(//tbody)[1]//div[@class="floatnone"]//a')
    for i in character_links:
        link = i.get_attribute('href')
        all_links.append({'link': link})

In [53]:
all_links

[{'link': 'https://lotr.fandom.com/wiki/Frodo_Baggins'},
 {'link': 'https://lotr.fandom.com/wiki/Sam_Gamgee'},
 {'link': 'https://lotr.fandom.com/wiki/Meriadoc_Brandybuck'},
 {'link': 'https://lotr.fandom.com/wiki/Peregrin_Took'},
 {'link': 'https://lotr.fandom.com/wiki/Gandalf'},
 {'link': 'https://lotr.fandom.com/wiki/Aragorn'},
 {'link': 'https://lotr.fandom.com/wiki/Legolas'},
 {'link': 'https://lotr.fandom.com/wiki/Gimli'},
 {'link': 'https://lotr.fandom.com/wiki/Boromir'},
 {'link': 'https://lotr.fandom.com/wiki/Tom_Bombadil'},
 {'link': 'https://lotr.fandom.com/wiki/Elrond'},
 {'link': 'https://lotr.fandom.com/wiki/Galadriel'},
 {'link': 'https://lotr.fandom.com/wiki/Th%C3%A9oden'},
 {'link': 'https://lotr.fandom.com/wiki/%C3%89omer'},
 {'link': 'https://lotr.fandom.com/wiki/%C3%89owyn'},
 {'link': 'https://lotr.fandom.com/wiki/Bilbo'},
 {'link': 'https://lotr.fandom.com/wiki/Faramir'},
 {'link': 'https://lotr.fandom.com/wiki/Denethor'},
 {'link': 'https://lotr.fandom.com/wiki/T

In [54]:
for d in all_links:
    driver.get(d["link"])
    driver.implicitly_wait(5)
    character_url = d["link"]
    try:
        character_other_names = driver.find_element(By.XPATH,'(//div[@class="pi-data-value pi-font"])[1]').text.strip()
    except:
        character_other_names = np.NaN
    try:
        character_description = driver.find_element(By.XPATH,'(//div[@id="mw-content-text"]//p)[5]').text.strip()
    except:
        character_description = np.NaN
    try:
        character_name = d["link"]
        character_name = character_name.split('/',4)[-1]
    except:
        character_name = np.NaN
    try:
        character_birth_date = driver.find_element(By.XPATH,'(//div[@class="pi-data-value pi-font"])[3]').text.strip()
    except:
        character_birth_date = np.NaN
    try:
        character_fameous_quote = driver.find_element(By.XPATH,'//div[@class="mw-parser-output"]//dl//dd[1]').text.strip()
    except:
        character_fameous_quote = np.NaN
    character_data.append({'url':character_url,
                            'desc':character_description,
                            'name':character_name,
                            'other_names':character_other_names,
                            'birth_date':character_birth_date,
                            'fameous_quote':character_fameous_quote})
    time.sleep(1)


In [56]:
df = pd.DataFrame(character_data)

In [57]:
df.head()

,url,desc,name,other_names,birth_date,fameous_quote
0,https://lotr.fandom.com/wiki/Frodo_Baggins,"Frodo Baggins, son of Drogo Baggins, was a hob...",Frodo_Baggins,"Frodo of the Nine Fingers, Nine-Fingered Frodo...","September 22, TA 2968 (SR 1368)","""I will take the Ring, though I do not know th..."
1,https://lotr.fandom.com/wiki/Sam_Gamgee,Sam admiring some flowers,Sam_Gamgee,"Sam, Perhael\nBanazîr Galbasi\nSamwise Gardner","April 6, TA 2980/SR 1380","""I know we are going to take a very long road,..."
2,https://lotr.fandom.com/wiki/Meriadoc_Brandybuck,Merry was born in TA 2982. He was the only chi...,Meriadoc_Brandybuck,"Merry, ""the Magnificent""\nKalimac ""Kali"" Brand...",TA 2982/SR 1382,"""But, but, lord, I offered you my sword. I do ..."
3,https://lotr.fandom.com/wiki/Peregrin_Took,Pippin was born to Paladin II Took and Eglanti...,Peregrin_Took,"Pippin,\nRazanur Tûk, (Westron name)\nErnil i ...",Spring TA 2990/SR 1390[1],"""No, my heart will not yet despair. Gandalf fe..."
4,https://lotr.fandom.com/wiki/Gandalf,,Gandalf,"Olórin, Mithrandir, Incánus, Tharkûn, Greyhame...",Before the Shaping of Arda,"""He wore a tall pointed blue hat, a long grey ..."


### Cleaning names as it will be needed to compare them to previously created variables: good_characters_names and bad_characters_names

In [59]:
df['name'] = df['name'].apply(lambda x: x.replace('Kham%C3%BBl','Khamûl'))
df['name'] = df['name'].apply(lambda x: x.replace('Durin%27s_Bane',"""Durin's Bane"""))
df['name'] = df['name'].apply(lambda x: x.replace('Th%C3%A9oden','Théoden'))
df['name'] = df['name'].apply(lambda x: x.replace('%C3%89omer','Éomer'))
df['name'] = df['name'].apply(lambda x: x.replace('%C3%89owyn','Éowyn'))
df['name'] = df['name'].apply(lambda x: x.replace('_',' '))

### Adding another column - check whether the character is a good or a bad one

In [66]:
def good_or_bad(x):
    if x in good_characters_names:
        return "Good"
    else:
        return "Bad"

In [68]:
df['good_or_bad_character'] = df['name'].apply(good_or_bad)

In [69]:
df

,url,desc,name,other_names,birth_date,fameous_quote,good_or_bad_character
0,https://lotr.fandom.com/wiki/Frodo_Baggins,"Frodo Baggins, son of Drogo Baggins, was a hob...",Frodo Baggins,"Frodo of the Nine Fingers, Nine-Fingered Frodo...","September 22, TA 2968 (SR 1368)","""I will take the Ring, though I do not know th...",Good
1,https://lotr.fandom.com/wiki/Sam_Gamgee,Sam admiring some flowers,Sam Gamgee,"Sam, Perhael\nBanazîr Galbasi\nSamwise Gardner","April 6, TA 2980/SR 1380","""I know we are going to take a very long road,...",Good
2,https://lotr.fandom.com/wiki/Meriadoc_Brandybuck,Merry was born in TA 2982. He was the only chi...,Meriadoc Brandybuck,"Merry, ""the Magnificent""\nKalimac ""Kali"" Brand...",TA 2982/SR 1382,"""But, but, lord, I offered you my sword. I do ...",Good
3,https://lotr.fandom.com/wiki/Peregrin_Took,Pippin was born to Paladin II Took and Eglanti...,Peregrin Took,"Pippin,\nRazanur Tûk, (Westron name)\nErnil i ...",Spring TA 2990/SR 1390[1],"""No, my heart will not yet despair. Gandalf fe...",Good
4,https://lotr.fandom.com/wiki/Gandalf,,Gandalf,"Olórin, Mithrandir, Incánus, Tharkûn, Greyhame...",Before the Shaping of Arda,"""He wore a tall pointed blue hat, a long grey ...",Good
5,https://lotr.fandom.com/wiki/Aragorn,"Aragorn as a toddler, portrayed by Luke Johnst...",Aragorn,"Estel, Thorongil,\nElessar / Edhelharn,\nTelco...","March 1, TA 2931","""I am Aragorn son of Arathorn, and am called E...",Good
6,https://lotr.fandom.com/wiki/Legolas,Legolas was a Sindar Elf who was part of the F...,Legolas,Greenleaf[1],Departed to Aman in FO 120 from Ithilien,"""Nay, time does not tarry ever, but change and...",Good
7,https://lotr.fandom.com/wiki/Gimli,"Born in TA 2879, Gimli was the son of Glóin an...",Gimli,"Elf-friend, Lockbearer",TA 2879,"""Faithless is he that says farewell when the r...",Good
8,https://lotr.fandom.com/wiki/Boromir,Boromir in Osgiliath after its recapture,Boromir,"Captain of the White Tower,\nCaptain-General,\...","February 26, TA 3019 (aged 41)","""The Ring! Is it not a strange fate that we sh...",Good
9,https://lotr.fandom.com/wiki/Tom_Bombadil,"His domain was of modest size, but he seemed t...",Tom Bombadil,"Iarwain Ben-adar, Orald, Forn",Unknown (before the\ncoming of Melkor),"""Old Tom Bombadil is a merry fellow! Bright Bl...",Good
